In [152]:
using DataFrames, CSV, Dates, Measures, Statistics, StatsPlots, IterativeSolvers, Polynomials, DelimitedFiles
include("./src/OptimizerParaibaDoSul.jl")
OPS = OptimizerParaibaDoSul

Main.OptimizerParaibaDoSul

In [153]:
dir = "function_trials"
base_name = "base_results"
df_reservoir = DataFrame(CSV.File(joinpath(base_name,"results",base_name*"_reservoir_Hm3.csv")))[:,["month","ps_equivalent_reservoir"]]

,month,ps_equivalent_reservoir
,Int64,Float64
1,1,0.946673
2,2,0.998262
3,3,1.0
4,4,1.0
5,5,1.0
6,6,1.0
7,7,0.968483
8,8,0.913217
9,9,0.874887


In [154]:
gdf = groupby(df_reservoir, "month")
df_stats = combine(gdf,
    "ps_equivalent_reservoir" => mean => "mean",
    "ps_equivalent_reservoir" => maximum => "max",
    "ps_equivalent_reservoir" => minimum => "min",
    "ps_equivalent_reservoir" => std => "std",
    )

,month,mean,max,min,std
,Int64,Float64,Float64,Float64,Float64
1,1,0.83585,1.0,0.158549,0.197732
2,2,0.879866,1.0,0.204769,0.189582
3,3,0.905726,1.0,0.248081,0.168852
4,4,0.896392,1.0,0.225346,0.171108
5,5,0.876286,1.0,0.176443,0.175476
6,6,0.839564,1.0,0.122449,0.177403
7,7,0.790173,1.0,0.0506449,0.182603
8,8,0.730763,1.0,-0.0151182,0.183928
9,9,0.686146,1.0,-0.0292304,0.187882


In [155]:
function function_compiler(trial_params,df_stats,trial_name)
    confidence = 2
    b = [250,190,trial_params["min_flow"]]
    coef = zeros(12,3)
    for i in 1:size(df_stats,1)
        max = df_stats[i,"max"] - trial_params["reservoir_confidence"]*(df_stats[i,"max"] - df_stats[i,"mean"])
        min = df_stats[i,"mean"] - confidence*df_stats[i,"std"]
        avg = df_stats[i,"mean"]

        A = [max^2 max^1 max^0
            avg^2 avg^1 avg^0
            min^2 min^1 min^0]

        p = Polynomial(reverse(lsmr(A,b)))
        coef[i,:] = coeffs(p)
        d = derivative(p)
        x = -0.1:0.01:1
        y = [(p(i) > b[3]) && (d(i) > 0) ? p(i) : b[3] for i in x]
        p = plot(x,y,
            ylim = [0,300],
            legend = false,
            xlabel = "Reservatório Equivalente (%)",
            ylabel = "Vazão (m^3/s)",
            title = "Função de Defluência Mês $(i)"
        )

        p = plot!([avg], seriestype="vline",color=:red)
        annotate!(avg-0.07, 305, text("Média: "*string(round(avg,digits = 3)), :red, 6))

        p = plot!([160], seriestype="hline",color=:red)
        annotate!(-0.16, 160, text("160", :red, 6))

        p = plot!([0.8], seriestype="vline",color=:purple, alpha = 0.3)
        annotate!(0.8, -15, text("80%", :purple, 6))

        if !isdir(joinpath(trial_name,"figures"))
            mkdir(joinpath(trial_name,"figures"))
        end

        savefig(joinpath(trial_name,"figures//month_$(i).png"))
    end
    return coef
end

function_compiler (generic function with 1 method)

In [156]:
for min_flow in 130:10:170
    for reservoir_confidence in 0:0.1:0.5
        trial_params = Dict(
            "min_flow"=>min_flow,
            "reservoir_confidence"=>reservoir_confidence
        )
        trial_name = "trial_$(trial_params["min_flow"])_$(Int64(round(100*trial_params["reservoir_confidence"])))%"

        if !isdir(joinpath(dir,trial_name))
            mkdir(joinpath(dir,trial_name))
        end

        cp(joinpath(base_name), joinpath(dir,trial_name),force=true)
        rm(joinpath(dir,trial_name,"results"), recursive=true)

        coef = function_compiler(trial_params,df_stats,joinpath(dir,trial_name))
        writedlm(joinpath(dir,trial_name,"defluence_poly.csv"),  coef, '\t')
        writedlm(joinpath(dir,trial_name,"defluence_poly_meta.csv"),  [min_flow, reservoir_confidence], '\t')

        OPS.run_simulation(joinpath(dir,trial_name))
    end
end

########### Simulation Start ###########
Simulation progress: 0.09363295880149813%
Simulation progress: 0.18726591760299627%
Simulation progress: 0.2808988764044944%
Simulation progress: 0.37453183520599254%
Simulation progress: 0.4681647940074906%
Simulation progress: 0.5617977528089888%
hm3_per_month_to_m3_per_sec(inflow, month) = 199.16557921838952
hm3_per_month_to_m3_per_sec(inflow, month) + function_flow_value = 449.165579218017
hm3_per_month_to_m3_per_sec(inflow, month) = 249.99999999962756
Simulation progress: 0.6554307116104869%
hm3_per_month_to_m3_per_sec(inflow, month) = 116.0
hm3_per_month_to_m3_per_sec(inflow, month) + function_flow_value = 365.18736144920746
hm3_per_month_to_m3_per_sec(inflow, month) = 249.18736144920751
Simulation progress: 0.7490636704119851%
hm3_per_month_to_m3_per_sec(inflow, month) = 119.0
hm3_per_month_to_m3_per_sec(inflow, month) + function_flow_value = 359.8877000943977
hm3_per_month_to_m3_per_sec(inflow, month) = 240.88770009439764
Simulation prog